# Phase 2 (recommendation links) - Ongoing

## NEXT: 
- 1 hyperlinks
- create a functions to calcualte precentage of change by area 
- 2 send by emaiL

In [ ]:
# Dependencies

import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver 
import time
import seaborn as sns 
from Hemnet import preprocessing,hemnet_generator, dep_filter,pct_change_metric
from Scraper import url_extractor , scraper2
from bs4 import BeautifulSoup
import requests
from itertools import chain 
import numpy as np 
import re  
from flask import Flask, request,render_template
import pickle

In [ ]:
#zones = [18027,18028,18042,17951]
#for area_code in zones:
    #pct_change_metric (area_code , num_pages= 50, metric = 'pris_per_m2')

In [ ]:
#links = []
#for link in soup.findAll('a', attrs={'href': re.compile("^https://www.hemnet")}):
    #links.append(link.get('href') )

# Model deployment

## 1. Flask app 

## NEXT: 

- Continue with docker
- learn more css and html
- choose the same structure and improve the design
- choose how to distribute the funtions 
- how to display tables
- Add info to the pipeline
- https://github.com/kishan0725/AJAX-Movie-Recommendation-System-with-Sentiment-Analysis

In [5]:
# Create a Pickle file using serialization - This step is on the notebook  

pickle_out1 = open("scraper2.pkl","wb") # this file should be on my folder 
pickle.dump(scraper2, pickle_out1) # classifier is the name of the model 

pickle_out2 = open("url_extractor.pkl","wb") 
pickle.dump(url_extractor, pickle_out2) 

pickle_out3 = open("pct_change_metric.pkl","wb") 
pickle.dump(pct_change_metric, pickle_out3) 

pickle_out1.close()
pickle_out2.close()
pickle_out3.close()

In [ ]:
# 1 create the flask env
app = Flask(__name__)
#Swagger(app)

# 2 Open the file 
pickle_in1 = open("scraper2.pkl","rb")
scraper2 = pickle.load(pickle_in1)

pickle_in2 = open("url_extractor.pkl","rb")
url_extractor = pickle.load(pickle_in2)

pickle_in3 = open("pct_change_metric.pkl","rb")
pct_change_metric = pickle.load(pickle_in3)

@app.route('/')
def welcome():
    return render_template('index.html')

@app.route('/predict',methods=["GET"])
def predict():
    
    # Remember to add new parameters here: 
    area = request.args.get("area")
    keys = request.args.get("keys")
    min_year = request.args.get("min_year")
    current_url = request.args.get("current_url")
    relevant_only = request.args.get("relevant_only")
    sold_age = request.args.get("sold_age")
    loan_limit = request.args.get("loan_limit")

    current_url_new = url_extractor(area, keys, min_year )

    df = scraper2(current_url = current_url_new ,relevant_only = relevant_only ,sold_age = sold_age,loan_limit = int(loan_limit))

    
    return render_template('view.html',tables=[df.to_html(classes ='scraper')],
    titles = ['na', 'Results'])

@app.route('/comparison',methods=["GET"])
def compare_pct():
 
    # Remember to add new parameters here: 
     
    area = request.args.get("area")
    num_pages = request.args.get("num_pages")
    metric = request.args.get("metric")

    output = pct_change_metric(area = area, num_pages = num_pages , metric = metric )

    return output

if __name__=='__main__':
    app.run()
    

# if __name__=='__main__':
#     app.run(host='0.0.0.0',port=8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


## 2. Docker
https://www.youtube.com/watch?v=cDwsaQoP4Lk&list=PLZoTAELRMXVNKtpy0U_Mx9N26w8n0hIbs&index=6 

# GitHub pull request

In [ ]:
! pwd

In [ ]:
import datetime

# 2 Pull request
! git pull

# 3
now = datetime.datetime.now()
commit_message = "Last run on " + str(now)
! cd '/Users/Tabe/Desktop/Courses/Data-Science /Test-Projects/Python-Projects/HemnetScraper'
! echo $commit_message > commit_message.txt
! git add . 
! git commit -F commit_message.txt
! git push origin master # here I choose either master or branch 
print ('Done!')